a. Extraer la información del archivo.

In [104]:
# Conectar con unidad de drive donde esta guardado el csv con los datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
import pandas as pd

# Leer el archivo con pandas
file_path = '/content/drive/MyDrive/certificado DS/country_vaccinations.csv'
df = pd.read_csv(file_path)

b. Mostrar la estructura y tipos de datos de cada columna para identificar qué operaciones puedes realizar con cada una de ellas, asegurándote que las columnas con fechas sean del tipo datetime64.

In [106]:
# Convertir la columna de fecha a datetime64
df['date'] = pd.to_datetime(df['date'])

# Mostrar tipos de datos de cada columna
print(df.dtypes)

country                                        object
iso_code                                       object
date                                   datetime64[ns]
total_vaccinations                            float64
people_vaccinated                             float64
people_fully_vaccinated                       float64
daily_vaccinations_raw                        float64
daily_vaccinations                            float64
total_vaccinations_per_hundred                float64
people_vaccinated_per_hundred                 float64
people_fully_vaccinated_per_hundred           float64
daily_vaccinations_per_million                float64
vaccines                                       object
source_name                                    object
source_website                                 object
dtype: object


c. Determinar la cantidad de vacunas aplicadas de cada compañía (con base en cómo lo reporta cada país en la columna vaccines, en otras palabras, agrupe por vaccines y realice la sumatoria).

In [112]:
# Muchas de las filas reportan que se usaron vacunas de varias companias
# y no se distingue cuanto del total le corresponde a cada una.

# Aunque en Kaggle hay un archivo que si separa las vacunas por fabricante
# (country_vaccinations_by_manufactures.csv), no tiene todos los datos.

# Entonces aqui se **asume** que para cada registro el total de vacunas esta
# repartido en partes iguales entre cada compania

from collections import defaultdict

vaccines_per_company = defaultdict(float)

for index, row in df.iterrows():
  if pd.isna(row['daily_vaccinations']):
    continue

  company_count = len(row['vaccines'].split(','))
  for company in row['vaccines'].split(','):
    vaccines_per_company[company.strip()] += row['daily_vaccinations'] / company_count

print("Cantidad de vacunas aplicadas por cada compañia:")
for company, count in vaccines_per_company.items():
  print(f"{company}: {int(count)}")

print(f"Total de vacunas: {sum(vaccines_per_company.values())}")

Cantidad de vacunas aplicadas por cada compañia:
Johnson&Johnson: 798188049
Oxford/AstraZeneca: 1788508232
Pfizer/BioNTech: 1327063777
Sinopharm/Beijing: 1097884739
Sinovac: 1141941598
Sputnik V: 953451825
Moderna: 975696177
CanSino: 755070862
Novavax: 156250657
Sputnik Light: 34938495
Covaxin: 662061762
Sinopharm/Wuhan: 654809418
ZF2001: 657620058
Abdala: 52587589
Soberana Plus: 11267652
Soberana02: 35213471
COVIran Barekat: 16261890
FAKHRAVAC: 16261890
Razi Cov Pars: 16261890
SpikoGen: 16261890
QazVac: 8239296
EpiVacCorona: 82901304
Medigen: 12335896
Turkovac: 49161442
Total de vacunas: 11320239870.99997



d. Obtener la cantidad de vacunas aplicadas en todo el mundo.


In [113]:
world_vaccinations_total = df['daily_vaccinations'].sum()
print(f"Cantidad de vacunas aplicadas en todo el mundo: {world_vaccinations_total}")
# seems legit https://www.bloomberg.com/graphics/covid-vaccine-tracker-global-distribution/

Cantidad de vacunas aplicadas en todo el mundo: 11320239871.0


e. Calcular el promedio de vacunas aplicadas por país.

In [114]:
total_vaccinations_per_country = df.groupby('country')['daily_vaccinations'].sum()
print(f"Promedio de vacunas aplicadas por pais: {total_vaccinations_per_country.mean()}")

Promedio de vacunas aplicadas por pais: 50763407.49327354


f. Determinar la cantidad de vacunas aplicadas el día 29/01/21 en todo el mundo.

In [115]:
specific_date = pd.Timestamp('2021-01-29')
rows_in_date = df[df['date'] == specific_date]
vaccines_applied_in_date = rows_in_date['daily_vaccinations'].sum()
print(f"Vacunas aplicadas el día 29/01/21 en todo el mundo: {vaccines_applied_in_date}")

Vacunas aplicadas el día 29/01/21 en todo el mundo: 4884052.0


g. Crear un dataframe nuevo denominado conDiferencias que contenga los datos originales y una columna derivada (diferencias) con las diferencias de aplicación entre las columnas daily_vaccionations y daily_vaccionations_raw.

In [116]:
conDiferencias = df.copy()
conDiferencias['diferencias'] = conDiferencias['daily_vaccinations'] - conDiferencias['daily_vaccinations_raw']

# Checar algunos registros sin valores NaN en las columnas usadas
sample = conDiferencias[
      conDiferencias['daily_vaccinations'].notna() &
      conDiferencias['daily_vaccinations_raw'].notna()
    ]
sample = sample = sample[['diferencias', 'daily_vaccinations', 'daily_vaccinations_raw']]
print(sample.head(5))

# daily_vaccinations y daily_vaccinations_raw, que representan estas columnas?
# Kaggle les da la misma descripcion

     diferencias  daily_vaccinations  daily_vaccinations_raw
94        3628.0              6487.0                  2859.0
101       1270.0              5285.0                  4015.0
339       2934.0              9802.0                  6868.0
397          3.0                63.0                    60.0
398        -12.0                66.0                    78.0


h. Obtener el periodo de tiempo entre el registro con fecha más reciente y el registro con fecha más antigua.

In [117]:
earliest_vaccination_date = df['date'].min()
latest_vaccination_date = df['date'].max()
print(f"Primera fecha: {earliest_vaccination_date}")
print(f"Ultima fecha: {latest_vaccination_date}")
print(f"Periodo de tiempo: {latest_vaccination_date - earliest_vaccination_date}")


Primera fecha: 2020-12-02 00:00:00
Ultima fecha: 2022-03-29 00:00:00
Periodo de tiempo: 482 days 00:00:00


i. Crear un dataframe nuevo denominado conCantidad que contenga los datos originales y una columna derivada (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna vaccines y separar por el carácter , ).

In [118]:
conCantidad = df.copy()
conCantidad['canVac'] = conCantidad['vaccines'].str.split(',').str.len()

# Checar algunos registros
sample = conCantidad[['canVac', 'vaccines']]
print(sample.head(5))

   canVac                                           vaccines
0       4  Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
1       4  Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
2       4  Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
3       4  Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
4       4  Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...



j. Generar un dataframe denominado antes20 con todos los registros que se hayan realizado antes del 20 de diciembre de 2020.

In [119]:
specific_date = pd.Timestamp('2020-12-20')
antes20 = df[df['date'] < specific_date]

# Checar registros filtrados y la fecha mas avanzada que se incluye
print(antes20['date'])
print(f"Ultima fecha en antes20: {antes20['date'].max()}")


13403   2020-12-14
13404   2020-12-15
13405   2020-12-16
13406   2020-12-17
13407   2020-12-18
           ...    
82362   2020-12-15
82363   2020-12-16
82364   2020-12-17
82365   2020-12-18
82366   2020-12-19
Name: date, Length: 70, dtype: datetime64[ns]
Ultima fecha en antes20: 2020-12-19 00:00:00



k. Obtener un dataframe denominado pfizer con todos los registros donde se haya utilizado la vacuna Pfizer.


In [120]:
VACCINE_NAME = 'pfizer'
pfizer = df[df['vaccines'].str.contains(VACCINE_NAME, case=False, na=False)]

# Checar registros filtrados
print(pfizer['vaccines'])

0        Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
1        Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
2        Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
3        Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
4        Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...
                               ...                        
85069         Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
85070         Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
85071         Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
85072         Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
85073         Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
Name: vaccines, Length: 64193, dtype: object


l. Almacenar los dataframes generados (conDiferencias, conCantidad, antes20 y pfizer) en un archivo de Excel denominado resultadosReto.xlsx, donde cada dataframe ocupe una hoja diferente. Se recomienda ver la documentación de pd.ExcelWriter.

In [121]:
writer = pd.ExcelWriter('resultadosReto.xlsx', engine='openpyxl')

conDiferencias.to_excel(writer, sheet_name='conDiferencias', index=False)
conCantidad.to_excel(writer, sheet_name='conCantidad', index=False)
antes20.to_excel(writer, sheet_name='antes20', index=False)
pfizer.to_excel(writer, sheet_name='pfizer', index=False)

writer.close()

In [123]:
from google.colab import files

# Descargar archivo excel generado
files.download('resultadosReto.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>